In [ ]:
from prettytable import PrettyTable

class PageReplacement:
    def __init__(self, pages, frames, algorithm="FIFO"):
        """
        初始化页面置换模拟器
        
        参数:
        pages: 页面访问序列
        frames: 物理内存块数
        algorithm: 置换算法，可选 'FIFO'、'LRU' 或 'OPT'
        """
        self.pages = pages
        self.frames = frames
        self.algorithm = algorithm.upper()
        self.memory = [-1] * frames  # 初始化物理内存，-1表示空闲
        self.page_faults = 0
        self.hits = 0
        self.history = []  # 记录每一步的内存状态
        self.fault_history = []  # 记录每一步是否发生缺页
        
        # 算法特定数据结构
        if self.algorithm == "FIFO":
            self.queue = []  # FIFO队列
        elif self.algorithm == "LRU":
            self.last_used = {}  # 记录每个页面最后一次被访问的时间
        elif self.algorithm == "OPT":
            pass
    
    def simulate(self):
        """运行页面置换模拟"""
        for i, page in enumerate(self.pages):
            if self.algorithm == "FIFO":
                self._fifo_replace(i, page)
            elif self.algorithm == "LRU":
                self._lru_replace(i, page)
            elif self.algorithm == "OPT":
                self._opt_replace(i, page)
            else:
                raise ValueError("不支持的算法: " + self.algorithm)
            
            # 记录当前状态
            self.history.append(self.memory.copy())
        
        # 计算缺页率
        fault_rate = self.page_faults / len(self.pages) * 100
        
        return {
            "page_faults": self.page_faults,
            "fault_rate": fault_rate,
            "history": self.history,
            "fault_history": self.fault_history
        }
    
    def _opt_replace(self, time, page):
        """最佳(OPT)页面置换算法"""
        if page in self.memory:  # 页面已在内存中
            self.hits += 1
            self.fault_history.append(False)
            return
        
        self.page_faults += 1
        self.fault_history.append(True)
        
        # 检查是否有空闲帧
        if -1 in self.memory:
            index = self.memory.index(-1)
            self.memory[index] = page
        else:
            # 找出未来最长时间不会使用的页面
            future_indexes = {}
            
            # 对于内存中的每个页面，找出它在未来访问序列中最早出现的位置
            for p in self.memory:
                try:
                    next_use = self.pages[time+1:].index(p) + time + 1
                    future_indexes[p] = next_use
                except ValueError:  # 该页面在未来不会再被使用
                    future_indexes[p] = float('inf')
            
            # 选择未来最长时间不会使用的页面进行替换
            victim = max(future_indexes.items(), key=lambda x: x[1])[0]
            index = self.memory.index(victim)
            self.memory[index] = page
    
    def _fifo_replace(self, time, page):
        """FIFO页面置换算法"""
        if page in self.memory:  # 页面已在内存中
            self.hits += 1
            self.fault_history.append(False)
            return
        
        self.page_faults += 1
        self.fault_history.append(True)
        
        # 检查是否有空闲帧
        if -1 in self.memory:
            index = self.memory.index(-1)
            self.memory[index] = page
            self.queue.append(page)
        else:
            # 替换最早进入的页面
            victim = self.queue.pop(0)
            index = self.memory.index(victim)
            self.memory[index] = page
            self.queue.append(page)
    
    def _lru_replace(self, time, page):
        """LRU页面置换算法"""
        if page in self.memory:  # 页面已在内存中
            self.hits += 1
            self.last_used[page] = time  # 更新最后访问时间
            self.fault_history.append(False)
            return
        
        self.page_faults += 1
        self.fault_history.append(True)
        
        # 检查是否有空闲帧
        if -1 in self.memory:
            index = self.memory.index(-1)
            self.memory[index] = page
        else:
            # 找到最长时间未使用的页面
            lru_page = min(
                [p for p in self.memory], 
                key=lambda p: self.last_used.get(p, -1)
            )
            index = self.memory.index(lru_page)
            self.memory[index] = page
        
        # 更新最后访问时间
        self.last_used[page] = time
    
    def display_results(self):
        """使用PrettyTable展示模拟结果"""
        table = PrettyTable()
        
        # 添加列标题
        table.field_names = ["访问页面"] + [f"帧{i+1}" for i in range(self.frames)] + ["缺页"]
        
        # 添加每一步的状态
        for i, page in enumerate(self.pages):
            row = [page]
            
            for frame in self.history[i]:
                if frame == -1:
                    row.append("-")
                else:
                    row.append(frame)
            
            row.append("✓" if self.fault_history[i] else "")
            table.add_row(row)
        
        # 显示结果
        print(f"页面置换算法: {self.algorithm}")
        print(f"物理内存块数: {self.frames}")
        print(f"页面访问序列: {self.pages}")
        print(table)
        print(f"缺页次数: {self.page_faults}/{len(self.pages)}")
        print(f"缺页率: {self.page_faults/len(self.pages)*100:.2f}%")

def run_simulation(pages, frames, algorithm):
    simulator = PageReplacement(pages, frames, algorithm)
    simulator.simulate()
    simulator.display_results()

In [8]:
pages = [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
run_simulation(pages, 3, "FIFO")

页面置换算法: FIFO
物理内存块数: 3
页面访问序列: [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
+----------+-----+-----+-----+------+
| 访问页面 | 帧1 | 帧2 | 帧3 | 缺页 |
+----------+-----+-----+-----+------+
|    4     |  4  |  -  |  -  |  ✓   |
|    3     |  4  |  3  |  -  |  ✓   |
|    2     |  4  |  3  |  2  |  ✓   |
|    1     |  1  |  3  |  2  |  ✓   |
|    4     |  1  |  4  |  2  |  ✓   |
|    3     |  1  |  4  |  3  |  ✓   |
|    5     |  5  |  4  |  3  |  ✓   |
|    4     |  5  |  4  |  3  |      |
|    3     |  5  |  4  |  3  |      |
|    2     |  5  |  2  |  3  |  ✓   |
|    1     |  5  |  2  |  1  |  ✓   |
|    5     |  5  |  2  |  1  |      |
+----------+-----+-----+-----+------+
缺页次数: 9/12
缺页率: 75.00%


In [9]:
run_simulation(pages, 4, "FIFO")

页面置换算法: FIFO
物理内存块数: 4
页面访问序列: [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
+----------+-----+-----+-----+-----+------+
| 访问页面 | 帧1 | 帧2 | 帧3 | 帧4 | 缺页 |
+----------+-----+-----+-----+-----+------+
|    4     |  4  |  -  |  -  |  -  |  ✓   |
|    3     |  4  |  3  |  -  |  -  |  ✓   |
|    2     |  4  |  3  |  2  |  -  |  ✓   |
|    1     |  4  |  3  |  2  |  1  |  ✓   |
|    4     |  4  |  3  |  2  |  1  |      |
|    3     |  4  |  3  |  2  |  1  |      |
|    5     |  5  |  3  |  2  |  1  |  ✓   |
|    4     |  5  |  4  |  2  |  1  |  ✓   |
|    3     |  5  |  4  |  3  |  1  |  ✓   |
|    2     |  5  |  4  |  3  |  2  |  ✓   |
|    1     |  1  |  4  |  3  |  2  |  ✓   |
|    5     |  1  |  5  |  3  |  2  |  ✓   |
+----------+-----+-----+-----+-----+------+
缺页次数: 10/12
缺页率: 83.33%


In [10]:
run_simulation(pages, 3, "LRU")

页面置换算法: LRU
物理内存块数: 3
页面访问序列: [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
+----------+-----+-----+-----+------+
| 访问页面 | 帧1 | 帧2 | 帧3 | 缺页 |
+----------+-----+-----+-----+------+
|    4     |  4  |  -  |  -  |  ✓   |
|    3     |  4  |  3  |  -  |  ✓   |
|    2     |  4  |  3  |  2  |  ✓   |
|    1     |  1  |  3  |  2  |  ✓   |
|    4     |  1  |  4  |  2  |  ✓   |
|    3     |  1  |  4  |  3  |  ✓   |
|    5     |  5  |  4  |  3  |  ✓   |
|    4     |  5  |  4  |  3  |      |
|    3     |  5  |  4  |  3  |      |
|    2     |  2  |  4  |  3  |  ✓   |
|    1     |  2  |  1  |  3  |  ✓   |
|    5     |  2  |  1  |  5  |  ✓   |
+----------+-----+-----+-----+------+
缺页次数: 10/12
缺页率: 83.33%


In [ ]:
run_simulation(pages, 4, "LRU")

页面置换算法: LRU
物理内存块数: 4
页面访问序列: [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
+----------+-----+-----+-----+-----+------+
| 访问页面 | 帧1 | 帧2 | 帧3 | 帧4 | 缺页 |
+----------+-----+-----+-----+-----+------+
|    4     |  4  |  -  |  -  |  -  |  ✓   |
|    3     |  4  |  3  |  -  |  -  |  ✓   |
|    2     |  4  |  3  |  2  |  -  |  ✓   |
|    1     |  4  |  3  |  2  |  1  |  ✓   |
|    4     |  4  |  3  |  2  |  1  |      |
|    3     |  4  |  3  |  2  |  1  |      |
|    5     |  4  |  3  |  5  |  1  |  ✓   |
|    4     |  4  |  3  |  5  |  1  |      |
|    3     |  4  |  3  |  5  |  1  |      |
|    2     |  4  |  3  |  5  |  2  |  ✓   |
|    1     |  4  |  3  |  1  |  2  |  ✓   |
|    5     |  5  |  3  |  1  |  2  |  ✓   |
+----------+-----+-----+-----+-----+------+
缺页次数: 8/12
缺页率: 66.67%


In [12]:
run_simulation(pages, 3, "OPT")

页面置换算法: OPT
物理内存块数: 3
页面访问序列: [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
+----------+-----+-----+-----+------+
| 访问页面 | 帧1 | 帧2 | 帧3 | 缺页 |
+----------+-----+-----+-----+------+
|    4     |  4  |  -  |  -  |  ✓   |
|    3     |  4  |  3  |  -  |  ✓   |
|    2     |  4  |  3  |  2  |  ✓   |
|    1     |  4  |  3  |  1  |  ✓   |
|    4     |  4  |  3  |  1  |      |
|    3     |  4  |  3  |  1  |      |
|    5     |  4  |  3  |  5  |  ✓   |
|    4     |  4  |  3  |  5  |      |
|    3     |  4  |  3  |  5  |      |
|    2     |  2  |  3  |  5  |  ✓   |
|    1     |  1  |  3  |  5  |  ✓   |
|    5     |  1  |  3  |  5  |      |
+----------+-----+-----+-----+------+
缺页次数: 7/12
缺页率: 58.33%


In [13]:
run_simulation(pages, 4, "OPT")

页面置换算法: OPT
物理内存块数: 4
页面访问序列: [4, 3, 2, 1, 4, 3, 5, 4, 3, 2, 1, 5]
+----------+-----+-----+-----+-----+------+
| 访问页面 | 帧1 | 帧2 | 帧3 | 帧4 | 缺页 |
+----------+-----+-----+-----+-----+------+
|    4     |  4  |  -  |  -  |  -  |  ✓   |
|    3     |  4  |  3  |  -  |  -  |  ✓   |
|    2     |  4  |  3  |  2  |  -  |  ✓   |
|    1     |  4  |  3  |  2  |  1  |  ✓   |
|    4     |  4  |  3  |  2  |  1  |      |
|    3     |  4  |  3  |  2  |  1  |      |
|    5     |  4  |  3  |  2  |  5  |  ✓   |
|    4     |  4  |  3  |  2  |  5  |      |
|    3     |  4  |  3  |  2  |  5  |      |
|    2     |  4  |  3  |  2  |  5  |      |
|    1     |  1  |  3  |  2  |  5  |  ✓   |
|    5     |  1  |  3  |  2  |  5  |      |
+----------+-----+-----+-----+-----+------+
缺页次数: 6/12
缺页率: 50.00%
